In [ ]:
import configparser
from datetime import datetime, timedelta
from getpass import getuser
from os import listdir
from os.path import isdir, join
from pathlib import Path

import pandas as pd

readout_times = pd.read_csv(join("..", "csv", "readout_times.csv"))


def get_readout_time(hdr):
    df = readout_times
    readout_rate = hdr["READRATE"]
    binning = hdr["HBIN"]
    sub_image = hdr["FINALLIN"] - hdr["INITLIN"] + 1
    filter = (
        (df["Readout rate"] == readout_rate)
        & (df["Sub-image"] == sub_image)
        & (df["Binning"] == binning)
    )
    return df[filter]["Readout time"].values[0]


def calc_overhead_between_images(hdr, next_hdr):
    timestamp = datetime.fromisoformat(hdr["DATE-OBS"])
    next_timestamp = datetime.fromisoformat(next_hdr["DATE-OBS"])
    readout_time = get_readout_time(hdr)
    delay = next_timestamp - timestamp
    delay = delay.total_seconds() - hdr["EXPTIME"] - readout_time
    return delay


def read_config_file():
    sparc4_folder = join("C:\\", "Users", getuser(), "SPARC4", "ACS")
    cfg_file = join(sparc4_folder, "acs_config.cfg")
    cfg = configparser.ConfigParser()
    cfg.read(cfg_file)
    return cfg


def get_image_folder():
    cfg = read_config_file()
    today = Path(cfg.get("channel configuration", "image path").strip(r"\""))
    today_list = [file for file in listdir(today) if ".fits" in file]
    if today_list != []:
        return today
    yesterday = datetime.now() - timedelta(days=1)
    yesterday = join(today, "..", yesterday.strftime("%Y%m%d"))
    if not isdir(yesterday):
        raise FileNotFoundError(f"The folder {yesterday} does not exist.")
    return yesterday

In [ ]:
# VERIFY IF THERE IS ANY ZERO IN THE POLARIMETRIC MODE
import os
from os.path import join

import astropy.io.fits as fits
import numpy as np
import pandas as pd

folder = join("C:\\", "Users", "Denis", "Desktop", "tmp", "ch1")
files = os.listdir(folder)
headers = {"INSTMODE": [], "WPPOS": []}

for file in files:
    file_path = join(folder, file)
    hdr = fits.getheader(file_path)
    headers["INSTMODE"].append(hdr["INSTMODE"])
    headers["WPPOS"].append(hdr["WPPOS"])

headers = pd.DataFrame.from_dict(headers)
csv_file = join(folder, "..", "WPPOS.csv")
headers.to_csv(csv_file, index=False)

wrong_header = (headers["INSTMODE"] == "POLAR") & (headers["WPPOS"] == 0)
idxs = np.where(wrong_header)[0]
for idx in idxs:
    print(f"File {files[idx]} has a wrong header!")

In [ ]:
# GET SOME IMAGE HEADER KEYWORDS AND WRITEN THEM INTO A CSV
import os
from os.path import join

import astropy.io.fits as fits
import numpy as np
import pandas as pd

folder = get_image_folder()
files = [file for file in os.listdir(folder) if ".fits" in file]

headers = {
    "FILENAME": [],
    "DATE-OBS": [],
    "NCYCLES": [],
    "CYCLIND": [],
    "NSEQ": [],
    "SEQINDEX": [],
    "NFRAMES": [],
    "FRAMEIND": [],
    "EXPTIME": [],
    "OVERHEAD": [],
}

for idx, file in enumerate(files[:-1]):
    file_path = join(folder, file)
    hdr1 = fits.getheader(file_path)
    file_path = join(folder, files[idx + 1])
    hdr2 = fits.getheader(file_path)

    for key in headers.keys():
        if key == "OVERHEAD":
            continue
        headers[key].append(hdr1[key])
    headers["OVERHEAD"].append(calc_overhead_between_images(hdr1, hdr2))

headers = pd.DataFrame.from_dict(headers)
csv_file = join("keywords.csv")
headers.to_csv(csv_file, index=False)

In [ ]:
import re

kw_value = "+10:0"
kw_value = re.sub(r"^([+-]?\d{1,2}):(\d{1,2})$", r"\1:\2:00", kw_value)
h, m, s = kw_value.split(":")
n_digits = 2
if "-" in h:
    n_digits += 1
kw_value = f"{int(h):0{n_digits}}:{int(m):02}:{int(s):02}"
print(kw_value)

10:00:00


In [14]:
import astropy.units as u
from astropy.coordinates import Angle, SkyCoord

coord = Angle("-00:00:01")
coord < 0

UnitsError: No unit specified